In [2]:
# final_processor_split_folders.py
import pandas as pd
import re
import csv
import glob
import os
from openpyxl import Workbook

# ----------------------------
# Настройки
# ----------------------------
input_folder = "wildberries"       # папка с входными CSV
output_csv_folder = "final_csv"    # папка для CSV (и orphan)
output_xlsx_folder = "final_xlsx"  # папка для XLSX (СБИС)
db_csv = "database.csv"
db_xlsx = "database.xlsx"

# создаём папки
os.makedirs(output_csv_folder, exist_ok=True)
os.makedirs(output_xlsx_folder, exist_ok=True)

# ----------------------------
# Загружаем базу
# ----------------------------
if os.path.exists(db_xlsx):
    db = pd.read_excel(db_xlsx, dtype=str)
    print(f"Loaded database from {db_xlsx}")
elif os.path.exists(db_csv):
    db = pd.read_csv(db_csv, sep=";", dtype=str)
    print(f"Loaded database from {db_csv}")
else:
    raise FileNotFoundError("Не найден database.csv или database.xlsx")

db.columns = db.columns.str.strip().str.lower()
required_cols = {"gtin", "group", "price", "okei", "tax"}
if not required_cols.issubset(set(db.columns)):
    raise ValueError(f"В базе отсутствуют обязательные колонки: {required_cols}")

db["gtin"] = db["gtin"].astype(str).str.strip()

# ----------------------------
# Утилиты
# ----------------------------
def extract_gtin(kiz: str):
    if not isinstance(kiz, str):
        return None
    clean = kiz.strip().strip('"').strip("'")
    m = re.match(r"010(\d{13})", clean)
    return m.group(1) if m else None

def safe_float(x):
    try:
        return float(str(x).replace(",", "."))
    except Exception:
        return 0.0

def format_kiz_for_excel(kiz: str):
    s = str(kiz).replace('"', '""')
    return f'"{s}"'

# ----------------------------
# Сохранение
# ----------------------------
def save_final_csv(rows, out_csv_path):
    with open(out_csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerows(rows)

def save_final_xlsx(records_df, out_xlsx_path):
    wb = Workbook()
    ws = wb.active
    ws.title = "УПД"

    headers = ["№", "Наименование товара", "Кол-во", "Ед. изм.", "Цена", "Сумма", "НДС", "КИЗы"]
    ws.append(headers)

    row_num = 1
    for group, subset in records_df.groupby("group"):
        info = subset.iloc[0]
        kiz_values = subset["val"].tolist()
        kiz_count = len(kiz_values)
        price = safe_float(info["price"])
        total = price * kiz_count
        kiz_cell = ", ".join(format_kiz_for_excel(k) for k in kiz_values)

        ws.append([
            row_num,
            info["group"],
            kiz_count,
            info["okei"],
            price,
            total,
            info["tax"],
            kiz_cell
        ])
        row_num += 1

    wb.save(out_xlsx_path)

# ----------------------------
# Обработка файла
# ----------------------------
def process_file(file_path, db, csv_folder, xlsx_folder, report):
    file_name = os.path.basename(file_path)
    base_name = os.path.splitext(file_name)[0]

    print(f"\n--- Обработка {file_name} ---")

    df = pd.read_csv(file_path, sep=",", dtype=str, header=None)
    if df.shape[0] == 0:
        print("  Файл пуст — пропускаем.")
        report.append({"file": file_name, "total_kiz": 0, "matched": 0, "orphans": 0})
        return

    raw = df.iloc[0].dropna().tolist()
    kiz_row = [v for v in raw if isinstance(v, str) and re.match(r'^010\d{13}', v.strip().strip('"').strip("'"))]

    total_kiz = len(kiz_row)
    print(f"  Найдено КИЗов (после фильтра): {total_kiz}")

    records = []
    orphans = []

    for kiz in kiz_row:
        gtin = extract_gtin(kiz)
        if gtin:
            matched = db.loc[db["gtin"] == gtin]
            if not matched.empty:
                r = matched.iloc[0]
                records.append({
                    "val": kiz,
                    "gtin": gtin,
                    "group": r["group"],
                    "price": r["price"],
                    "okei": r["okei"],
                    "tax": r["tax"]
                })
            else:
                orphans.append((kiz, gtin))
        else:
            orphans.append((kiz, ""))

    matched_count = len(records)

    # CSV (для ЧЗ)
    rows = []
    row_no = 1
    if records:
        df_records = pd.DataFrame(records)
        for group, subset in df_records.groupby("group"):
            info = subset.iloc[0]
            kiz_values = subset["val"].tolist()
            kiz_count = len(kiz_values)
            row = [
                str(row_no),
                info["group"],
                info["price"],
                str(kiz_count),
                info["okei"],
                info["tax"],
                "КИЗ"
            ]
            row.extend(kiz_values)
            rows.append(row)
            row_no += 1

    out_csv = os.path.join(csv_folder, f"final_{base_name}.csv")
    if rows:
        save_final_csv(rows, out_csv)
        print(f"  Saved CSV: {out_csv}")
    else:
        print("  Нет сопоставленных КИЗов — CSV не создан.")

    # XLSX (для СБИС)
    out_xlsx = os.path.join(xlsx_folder, f"final_{base_name}.xlsx")
    if records:
        save_final_xlsx(pd.DataFrame(records), out_xlsx)
        print(f"  Saved XLSX: {out_xlsx}")
    else:
        print("  Нет сопоставленных КИЗов — XLSX не создан.")

    # Orphan
    orphan_path = os.path.join(csv_folder, f"final_{base_name}_orphan.csv")
    if orphans:
        with open(orphan_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f, delimiter=";", quotechar='"', quoting=csv.QUOTE_ALL)
            writer.writerow(["КИЗ", "GTIN"])
            for k, g in orphans:
                writer.writerow([k, g])
        print(f"  Saved orphan: {orphan_path}")
    else:
        print("  Нет orphan-кодов.")

    report.append({
        "file": file_name,
        "total_kiz": total_kiz,
        "matched": matched_count,
        "orphans": len(orphans),
        "csv_created": bool(rows),
        "xlsx_created": bool(records),
        "orphan_file": os.path.basename(orphan_path) if orphans else ""
    })

# ----------------------------
# Основной цикл
# ----------------------------
report = []
for f in glob.glob(os.path.join(input_folder, "*.csv")):
    process_file(f, db, output_csv_folder, output_xlsx_folder, report)

# ----------------------------
# Сводный отчёт
# ----------------------------
report_lines = []
total_kiz = total_matched = total_orphans = 0
report_lines.append("Файл | Всего КИЗ | Сопоставлено | Orphan | CSV | XLSX | OrphanFile")
for r in report:
    report_lines.append(f"{r['file']} | {r['total_kiz']} | {r['matched']} | {r['orphans']} | {int(r['csv_created'])} | {int(r['xlsx_created'])} | {r['orphan_file']}")
    total_kiz += r['total_kiz']
    total_matched += r['matched']
    total_orphans += r['orphans']

summary = f"ИТОГО: Всего КИЗ={total_kiz}, Сопоставлено={total_matched}, Orphan={total_orphans}"
report_lines.append("")
report_lines.append(summary)

report_path = os.path.join(output_csv_folder, "report.txt")
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(report_lines))

print("\n--- Сводный отчет ---")
print("\n".join(report_lines))
print(f"\nReport saved: {report_path}")


Loaded database from database.csv

--- Обработка 24.03.csv ---
  Найдено КИЗов (после фильтра): 752
  Saved CSV: final_csv/final_24.03.csv
  Saved XLSX: final_xlsx/final_24.03.xlsx
  Нет orphan-кодов.

--- Обработка 28.02.csv ---
  Найдено КИЗов (после фильтра): 542
  Saved CSV: final_csv/final_28.02.csv
  Saved XLSX: final_xlsx/final_28.02.xlsx
  Нет orphan-кодов.

--- Обработка 12.07.csv ---
  Найдено КИЗов (после фильтра): 475
  Saved CSV: final_csv/final_12.07.csv
  Saved XLSX: final_xlsx/final_12.07.xlsx
  Нет orphan-кодов.

--- Обработка 28.07.csv ---
  Найдено КИЗов (после фильтра): 331
  Saved CSV: final_csv/final_28.07.csv
  Saved XLSX: final_xlsx/final_28.07.xlsx
  Нет orphan-кодов.

--- Обработка 22.05.csv ---
  Найдено КИЗов (после фильтра): 658
  Saved CSV: final_csv/final_22.05.csv
  Saved XLSX: final_xlsx/final_22.05.xlsx
  Нет orphan-кодов.

--- Обработка 06.07.csv ---
  Найдено КИЗов (после фильтра): 1086
  Saved CSV: final_csv/final_06.07.csv
  Saved XLSX: final_xlsx/